## Web scraping for Data

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [2]:
Url = 'https://www.imdb.com/chart/top/'
Headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
res = requests.get(url = Url, headers = Headers)
soup = BeautifulSoup(res.content, 'html.parser')
#print(soup.prettify)
body = soup.find('tbody',attrs = {'class' : 'lister-list'})
#print(body.text)
title = body.findAll('a')
year = body.findAll('span',attrs = {'class' : 'secondaryInfo'})
years = [i.text.lstrip('(').rstrip(')') for i in year]
rating = body.findAll('td',attrs = {'class' : 'ratingColumn imdbRating'})
ratings = [i.text.strip('\n') for i in rating]
titles = []
links = []
movie_id = []
for i in tqdm(title):
    if(i.text != ' \n'):
        titles.append(i.text)
    for j in str(i):
        if(j == '?'):
            l = str(i).index(j)
            break
    s = str(i)[9:l]
    if s not in movie_id:
        movie_id.append(s)
    li = 'https://www.imdb.com'+s+'reviews?ref_=tt_urv'
    if( li not in links):
        links.append(li)
print(len(movie_id),len(links))

  0%|          | 0/500 [00:00<?, ?it/s]

250 250


In [3]:
data = {
    'Title' : titles,
    'Link' : links,
    'Year' : years,
    'Rating' : ratings,
    'Movie_ID' : movie_id
}
file = pd.DataFrame(data)

In [4]:
file.to_csv('C:/Users/joshy/Desktop/project/0_input/input.csv')

### Function to remove the files in a directory 

In [5]:
def remove_dir(direc):
    import os, shutil
    folder = direc
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

### Function for eliminating emojis and unwanted characters 

In [6]:
import re
import sys
import unidecode

# https://stackoverflow.com/a/49146722/330558
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

## Function for Web scraping and writing article data into text file

In [7]:
def extracter(url,id):
    URL = url
    id = id.replace(":","")
    print(id)
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    # Here the user agent is for Edge browser on windows 10. You can find your browser user agent from the above given link.
    r = requests.get(url=URL, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser') # If this line causes an error, run 'pip install html5lib' or install html5lib
    title = [i.text for i in soup.find_all('a', attrs = {'class' : 'title'})]
    review = [i.text for i in soup.find_all('div', attrs = {'class':'text show-more__control'})]
    user = [i.text.replace("*","") for i in soup.find_all('span', attrs = {'class' : 'display-name-link'})]
    # Directory
    directory = id
    # Parent Directory path
    parent_dir = "C:/Users/joshy/Desktop/project/2_data"
    # Path
    path = os.path.join(parent_dir, directory)
    # Create the directory
    if id not in os.listdir(parent_dir):
        os.mkdir(path)
    else:
        path
        
    for i in range(len(title)):
        title[i] = remove_emoji(title[i]) 
        title[i] = unidecode.unidecode(title[i])
        with open(f'{parent_dir}/{id}/{user[i]}.txt', 'a') as f:
            f.write(title[i])
            f.write('\n')
        
    for i in range(len(title)):
        review[i] = remove_emoji(review[i])
        review[i] = unidecode.unidecode(review[i])
        with open(f'{parent_dir}/{id}/{user[i]}.txt', 'a') as f:
            f.write(review[i])

In [8]:
path = "C:/Users/joshy/Desktop/project/2_data"
#remove_dir(path)
dir = os.listdir(path)
if(len(dir) > 0 and len(dir) < len(links)):
    for i in tqdm(range(len(dir)-1,len(links))):
        url = links[i]
        id = titles[i]
        extracter(url,id)
elif(len(dir) == len(links)):
    pass
else:
    for i in tqdm(range(len(dir),len(links))):
        url = links[i]
        id = titles[i]
        extracter(url,id)
    

In [9]:
import pandas as pd
import numpy as np

git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

In [236]:
file = pd.read_excel(r'C:\Users\joshy\Desktop\project\IMDB-Movie-Reviews-Large-Dataset-50k\train.xlsx')

In [237]:
file.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [238]:
rating = []
for i in file['Sentiment']:
  if i == 'pos':
    rating.append(1)
  elif i == 'neg':
    rating.append(-1)
  else:
    rating.append(0)
file['Rating'] = rating

In [239]:
file.head()

,Reviews,Sentiment,Rating
0,"When I first tuned in on this morning news, I ...",neg,-1
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg,-1
2,Why does this movie fall WELL below standards?...,neg,-1
3,Wow and I thought that any Steven Segal movie ...,neg,-1
4,"The story is seen before, but that does'n matt...",neg,-1


In [240]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [241]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to c:\users\joshy\appdata\local\temp\pip-req-build-knnjlrtn


  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git 'C:\Users\joshy\AppData\Local\Temp\pip-req-build-knnjlrtn'
  fatal: unable to access 'https://github.com/laxmimerit/preprocess_kgptalkie.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git 'C:\Users\joshy\AppData\Local\Temp\pip-req-build-knnjlrtn' did not run successfully.
  exit code: 128
  
  See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git 'C:\Users\joshy\AppData\Local\Temp\pip-req-build-knnjlrtn' did not run successfully.
exit code: 128

See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [242]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [17]:
file['Reviews'] = file ['Reviews'].apply(lambda x: get_clean(x))

C:\Users\joshy\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [18]:
tfidf = TfidfVectorizer(max_features = 5000)
X = file['Reviews']
y = file['Sentiment']

X = tfidf.fit_transform(X)

In [19]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845641 stored elements in Compressed Sparse Row format>

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 0)

In [21]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC()

In [22]:
y_pred = clf.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [69]:
x = "One of the finest films made in recent years. It's a poignant story about hope."

x = get_clean(x)
print(x)
vec = tfidf.transform([x])

one of the finest films made in recent years it is a poignant story about hope


In [25]:
vec.shape

(1, 5000)

In [26]:
clf.predict(vec)

array(['pos'], dtype=object)

## Extracting Data from text file into a Dictionary

In [27]:
import requests, zipfile, io, os
from tqdm.notebook import tqdm

In [28]:
folder_dict = {'test' : r'C:\Users\joshy\Desktop\project\2_data'}
text_dict = {'test' : {}}

In [29]:
for label, folder in folder_dict.items():
    movies = os.listdir(folder)
    for movie in tqdm(movies):
        text_files = os.listdir(os.path.join(folder, movie))
        for file in text_files:
            with open(os.path.join(folder, movie, file), 'r') as text_file:
                text_dict[label].setdefault(movie, []).append(' '.join(text_file.readlines()))

  0%|          | 0/250 [00:00<?, ?it/s]

In [30]:
text_dict['test']['Dead Poets Society'][0]

" Deserving 10/10\n \n A rare 10/10 and it's deserving of such. This movie had a profound impact on me....and no better person to play it then the great Robin Williams.He acted with such emotion and intensity and vigour in each scene. It is RARE to feel like I'm inside the film, but in this case I felt like Robin Williams was constantly talking to me in this film.Alongside him, the rest of the cast did a great job.This movie was more than just acting. It represented how a person should live and enjoy life. Thank you for such a gem."

In [31]:
text_dict['test']['Dead Poets Society'][1]

' A hymn to life\n \n "Dead Poets Society" is more than a lesson of life. It\'s a hymn to life. It has a good message: "Carpe diem", which means "Seize the day". I completely agree with that message because there\'s no coming back and time flies.It\'s a reasonable movie with an inspiring story, good message, great performances from the actors, wonderful sceneries, awesome soundtrack (classical music, for example), drama and comedy.John Keating is the kind of teacher that anyone would like to have as a teacher. A teacher with such a colorful imagination and sense of humor, a teacher who inspires his students to make their dreams come true and makes their lives extraordinary. I mean, wow! John Keating is brilliantly performed by the comedian Robin Williams. In general, Robin Williams is as humorous as usual in this movie, but we can also see some of his talent for drama.Besides this incredible teacher, other great characters are: Neil Perry, Knox Overstreet, Charlie Dalton, Steven Meeks,

In [32]:
import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [33]:
spacy_text = {}
for movie, text_list in tqdm(text_dict['test'].items()):
    spacy_text[movie] = list(nlp.pipe(text_list))

  0%|          | 0/250 [00:00<?, ?it/s]

In [228]:
df = pd.DataFrame({'Movies':list(text_dict['test'].keys())})

In [229]:
df.head()

,Movies
0,12 Angry Men
1,12 Years a Slave
2,1917
3,2001 A Space Odyssey
4,3 Idiots


In [230]:
def scores_no_rev(lis):
    score = []
    for j in tqdm(lis):
        s = 0
        rev = [get_clean(k) for k in j.split(".")]
        for sen in rev:
            vec = tfidf.transform([sen])
            if(clf.predict(vec)[0] == 'pos'):
                s += 1
            elif(clf.predict(vec)[0] == 'neg'):
                s -= 1
            else:
                continue
        score.append(s/len(rev))
        n_rev = len(lis)
    return score,n_rev 

In [231]:
scores = []
no_rev  =[]
for i in tqdm(list(text_dict['test'].keys())):
    score,n_rev = scores_no_rev(text_dict['test'][i])
    scores.append(sum(score))
    no_rev.append(n_rev)

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

C:\Users\joshy\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [232]:
df['Scores'] = scores
df['No.of.Reviews'] = no_rev                

In [233]:
df['Rating'] = df['Scores']*10/df['No.of.Reviews']

In [255]:
f = pd.read_csv(r"C:\Users\joshy\Desktop\project\0_input\input.csv")
sorted = f.sort_values(by='Title')
sorted.head()

,Unnamed: 0,Title,Link,Year,Rating,Movie_ID
4,4,12 Angry Men,https://www.imdb.com/title/tt0050083/reviews?r...,1957,8.9,/title/tt0050083/
182,182,12 Years a Slave,https://www.imdb.com/title/tt2024544/reviews?r...,2013,8.1,/title/tt2024544/
123,123,1917,https://www.imdb.com/title/tt8579674/reviews?r...,2019,8.2,/title/tt8579674/
88,88,2001: A Space Odyssey,https://www.imdb.com/title/tt0062622/reviews?r...,1968,8.3,/title/tt0062622/
86,86,3 Idiots,https://www.imdb.com/title/tt1187043/reviews?r...,2009,8.3,/title/tt1187043/


In [256]:
df['Actual Rating'] = f['Rating']

In [257]:
df.head()

,Movies,Scores,No.of.Reviews,Rating,Actual Rating
0,12 Angry Men,3.785693,25,1.514277,9.2
1,12 Years a Slave,3.979807,25,1.591923,9.2
2,1917,5.576931,25,2.230773,9.0
3,2001 A Space Odyssey,5.568211,25,2.227284,9.0
4,3 Idiots,2.089509,25,0.835804,8.9


In [227]:
df.to_csv(f'C:/Users/joshy/Desktop/project/output.csv') 

In [124]:
dicti = pd.read_csv(r'C:\Users\joshy\Desktop\project\Loughran-McDonald_MasterDictionary_1993-2021.csv')
dicti.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
0,AARDVARK,1,354,1.550080e-08,1.422600e-08,3.815486e-06,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.313627e-10,8.653817e-12,9.241714e-09,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.940882e-10,1.169679e-10,5.290465e-08,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.269840e-09,6.654735e-10,1.595100e-07,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,3.752595e-07,3.809464e-07,3.529356e-05,1108,0,0,0,0,0,0,0,3,12of12inf


### Getting Positive words from the Master Dictionary 

In [125]:
pos = dicti[dicti['Positive'] > 0]
pos = [word.lower() for word in pos.Word]
pos

['able',
 'abundance',
 'abundant',
 'acclaimed',
 'accomplish',
 'accomplished',
 'accomplishes',
 'accomplishing',
 'accomplishment',
 'accomplishments',
 'achieve',
 'achieved',
 'achievement',
 'achievements',
 'achieves',
 'achieving',
 'adequately',
 'advancement',
 'advancements',
 'advances',
 'advancing',
 'advantage',
 'advantaged',
 'advantageous',
 'advantageously',
 'advantages',
 'alliance',
 'alliances',
 'assure',
 'assured',
 'assures',
 'assuring',
 'attain',
 'attained',
 'attaining',
 'attainment',
 'attainments',
 'attains',
 'attractive',
 'attractiveness',
 'beautiful',
 'beautifully',
 'beneficially',
 'benefited',
 'benefiting',
 'benefitted',
 'benefitting',
 'best',
 'better',
 'bolstered',
 'bolstering',
 'bolsters',
 'boom',
 'booming',
 'boost',
 'boosted',
 'breakthrough',
 'breakthroughs',
 'brilliant',
 'charitable',
 'collaborate',
 'collaborated',
 'collaborates',
 'collaborating',
 'collaboration',
 'collaborations',
 'collaborative',
 'collaborator'

### Getting Negative words from the Master Dictionary 

In [126]:
neg = dicti[dicti['Negative'] > 0]
neg = [word.lower() for word in neg.Word]
neg

['abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abandonments',
 'abandons',
 'abdicated',
 'abdicates',
 'abdicating',
 'abdication',
 'abdications',
 'aberrant',
 'aberration',
 'aberrational',
 'aberrations',
 'abetting',
 'abnormal',
 'abnormalities',
 'abnormality',
 'abnormally',
 'abolish',
 'abolished',
 'abolishes',
 'abolishing',
 'abrogate',
 'abrogated',
 'abrogates',
 'abrogating',
 'abrogation',
 'abrogations',
 'abrupt',
 'abruptly',
 'abruptness',
 'absence',
 'absences',
 'absenteeism',
 'abuse',
 'abused',
 'abuses',
 'abusing',
 'abusive',
 'abusively',
 'abusiveness',
 'accident',
 'accidental',
 'accidentally',
 'accidents',
 'accusation',
 'accusations',
 'accuse',
 'accused',
 'accuses',
 'accusing',
 'acquiesce',
 'acquiesced',
 'acquiesces',
 'acquiescing',
 'acquit',
 'acquits',
 'acquittal',
 'acquittals',
 'acquitted',
 'acquitting',
 'adulterate',
 'adulterated',
 'adulterating',
 'adulteration',
 'adulterations',
 'adversarial',
 'adversaries',
 

### Getting Complex words from the Master Dictionary 

In [127]:
com = dicti[dicti['Syllables'] > 2]
complex_words = [word.lower() for word in com.Word]
complex_words

['abaci',
 'abacus',
 'abacuses',
 'abalone',
 'abalones',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abandonments',
 'abandons',
 'abasement',
 'abasements',
 'abases',
 'abashedly',
 'abashes',
 'abashing',
 'abashment',
 'abashments',
 'abasing',
 'abated',
 'abatement',
 'abatements',
 'abating',
 'abattoir',
 'abattoirs',
 'abbesses',
 'abbreviate',
 'abbreviated',
 'abbreviates',
 'abbreviating',
 'abbreviation',
 'abbreviations',
 'abdicate',
 'abdicated',
 'abdicates',
 'abdicating',
 'abdication',
 'abdications',
 'abdomen',
 'abdomens',
 'abdominal',
 'abdominals',
 'abducted',
 'abducting',
 'abduction',
 'abductions',
 'abductor',
 'abductors',
 'aberrant',
 'aberration',
 'aberrational',
 'aberrations',
 'abetted',
 'abetter',
 'abetters',
 'abetting',
 'abettor',
 'abettors',
 'abeyance',
 'abeyances',
 'abhorrence',
 'abhorrences',
 'abhorrent',
 'abhorrently',
 'abhorring',
 'abidance',
 'abidances',
 'abided',
 'abiding',
 'abidingly',
 'abilities',
 'a

In [157]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

### Tokenizing the words and sentences using ntlk module 

### Cleaning Data using nltk stopwords and isalpha() 

### Calculating Positive score, Negative score, Polarity score, Subjectivity score 

In [164]:
def clean_words(lstr):
    token_word = {}
    token_sent ={}
    for i in range(len(lstr)):
        token_word[i] = word_tokenize(lstr[i])
        token_sent[i] = sent_tokenize(lstr[i])
    #print(token_word)  
    #print(token_sent[3])
    clean_token_word = {}
    clean_token_sent = {}
    for i in range(len(token_word)):
        clean_token_word[i] = [word for word in token_word[i] if word.lower() not in stop_words and word.isalpha() == True]
        sents=[]
        for j in token_sent[i]:
            sent = " ".join([wor for wor in word_tokenize(j) if wor.lower() not in stop_words and wor.isalpha() == True ])
            sents.append(sent.strip())
        clean_token_sent[i] = sents
    #print(clean_token_sent[7])
    #print(clean_token_word[7][:15])
    pos_score = []
    neg_score = []
    for i in range(len(clean_token_word)):
        pos_score.append(len([word for word in clean_token_word[i] if word in pos]))
        neg_score.append(len([wor for wor in clean_token_word[i] if wor in neg]))
    polarity_score = [((pos_score[i] - neg_score[i])/((pos_score[i] + neg_score[i])+0.000001)) for i in range(len(pos_score))]
    subjectivity_score = [((pos_score[i] + neg_score[i])/((len(clean_token_word[i]))+0.000001)) for i in range(len(pos_score))]
    return pos_score,neg_score,polarity_score,subjectivity_score

In [174]:
positive_score = {}
negative_score = {}
polarity_Score = {}
Subjectivity_Score = {}
for i in tqdm(list(text_dict['test'].keys())):
    p,n,q,s = clean_words(text_dict['test'][i])
    positive_score[i]=p
    negative_score[i]=n
    polarity_Score[i]=q
    Subjectivity_Score[i]=s

  0%|          | 0/250 [00:00<?, ?it/s]

In [175]:
positive_score['12 Angry Men']

[1, 13, 3, 3, 13, 0, 2, 2, 1, 5, 2, 1, 2, 0, 3, 0, 4, 7, 4, 2, 5, 6, 4, 3, 13]

In [180]:
import syllapy

## Function to calculate FOG Index,Word count,Complex Word Count....        

In [181]:
def calculate_fog(document,i):
    doc = nlp(document, disable=['tagger', 'ner', 'entity_linker', 'textcat', 'entitry_ruler'])
    sen_list = list(doc.sents)
    num_sen = len(clean_token_sent[i])
    num_words = 0
    num_complex_words = 0
    num_syllable = 0
    num_char = 0
    for sen_obj in sen_list:
        words_in_sen = [token.text for token in sen_obj if token.is_alpha]
        num_words += len(words_in_sen)
        num_complex  = 0
        for word in words_in_sen:
            num_char += len(word.strip())
            if 'es' not in word and 'ed' not in word:
                num_syl = syllapy.count(word.lower())
                num_syllable += num_syl
            else:
                num_syl = syllapy.count(word.lower())-1
                num_syllable += num_syl
            if num_syl > 2:
                num_complex += 1
        num_complex_words += num_complex
    
    avg_words_per_sen = round((num_words / num_sen),4)
    pern_comp_words = round((num_complex_words / num_words)*100,4)
    fog = 0.4 * (avg_words_per_sen + (pern_comp_words*100))
    fog = round(fog,4)
    avg_word_length = round((num_char/num_words),4)
    res = [avg_words_per_sen,pern_comp_words,fog,avg_words_per_sen,num_complex_words,num_words,num_syllable,avg_word_length]
    return res;

### Writing Columns of Output csv file  

In [259]:
for a in tqdm(list(text_dict['test'].keys())):
    token_word = {}
    token_sent ={}
    for i in range(len(text_dict['test'][a])):
        token_word[i] = word_tokenize(text_dict['test'][a][i])
        token_sent[i] = sent_tokenize(text_dict['test'][a][i])
    #print(token_word)  
    #print(token_sent[3])
    clean_token_word = {}
    clean_token_sent = {}
    for i in range(len(token_word)):
        clean_token_word[i] = [word for word in token_word[i] if word.lower() not in stop_words and word.isalpha() == True]
        sents=[]
        for j in token_sent[i]:
            sent = " ".join([wor for wor in word_tokenize(j) if wor.lower() not in stop_words and wor.isalpha() == True ])
            sents.append(sent.strip())
        clean_token_sent[i] = sents
    #print(clean_token_sent)
    #print(clean_token_word[7][:15])
    result_data = []
    for i in range(len(positive_score[a])):
        result_dict={}
        result = calculate_fog("".join(clean_token_sent[i]),i)
        keys = ['AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','AVG WORD LENGTH']
        for j in range(len(keys)):
            result_dict[keys[j]] = result[j]
        result_data.append(result_dict)
    data = [i[:-4] for i in os.listdir(f"C:/Users/joshy/Desktop/project/2_data/{a}")]
    dff = pd.DataFrame({'Users':data})
    dff['POSITIVE SCORE'] = positive_score[a]
    dff['NEGATIVE SCORE'] = negative_score[a]
    dff['POLARITY SCORE'] = polarity_Score[a]
    dff['SUBJECTIVITY SCORE'] = Subjectivity_Score[a]
    output_index = ['AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','AVG WORD LENGTH']
    for i in range(len(output_index)):
        key = output_index[i]
        dff[key] = [j[key] for j in result_data]
    #dff['Overall_impression'] = ['Positive' for i in dff['POSITIVE SCORE'] if dff['POSITIVE SCORE'] > dff['NEGATIVE SCORE'] else 'Negative']
    dff.to_csv(f'C:/Users/joshy/Desktop/project/output/{a}.csv')

  0%|          | 0/250 [00:00<?, ?it/s]

C:\Users\joshy\AppData\Roaming\Python\Python39\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
C:\Users\joshy\AppData\Roaming\Python\Python39\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
C:\Users\joshy\AppData\Roaming\Python\Python39\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or '